## The Dataset 

The original source of the data is Australian Government's Bureau of Meteorology and the latest data can be gathered from http://www.bom.gov.au/climate/dwo/.

This dataset contains observations of weather metrics for each day from 2008 to 2017. The weatherAUS.csv is the dataset that will be used for this project.

In [238]:
# importing the libraries and installing packages.

def warn (*args, **kwargs):
    pass
import warnings
warnings.warn = warn

In [239]:
import piplite
await piplite.install(['pandas'])
await piplite.install(['numpy'])

In [240]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import jaccard_score
from sklearn.metrics import f1_score
from sklearn.metrics import log_loss
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix, accuracy_score
import sklearn.metrics as metrics
from sklearn import svm

In [241]:
from pyodide.http import pyfetch

async def download(url, filename):
    response = await pyfetch(url)
    if response.status == 200:
        with open(filename, 'wb') as f:
            f.write(await response.bytes())

In [242]:
path = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-ML0101EN-SkillUp/labs/ML-FinalAssignment/Weather_Data.csv'

In [243]:
await download(path, 'Weather_Data.csv')
filename ='Weather_Data.csv'

In [244]:
df = pd.read_csv('Weather_Data.csv')
df.head()

,Date,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,WindDir3pm,...,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow
0,2/1/2008,19.5,22.4,15.6,6.2,0.0,W,41,S,SSW,...,92,84,1017.6,1017.4,8,8,20.7,20.9,Yes,Yes
1,2/2/2008,19.5,25.6,6.0,3.4,2.7,W,41,W,E,...,83,73,1017.9,1016.4,7,7,22.4,24.8,Yes,Yes
2,2/3/2008,21.6,24.5,6.6,2.4,0.1,W,41,ESE,ESE,...,88,86,1016.7,1015.6,7,8,23.5,23.0,Yes,Yes
3,2/4/2008,20.2,22.8,18.8,2.2,0.0,W,41,NNE,E,...,83,90,1014.2,1011.8,8,8,21.4,20.9,Yes,Yes
4,2/5/2008,19.7,25.7,77.4,4.8,0.0,W,41,NNE,W,...,88,74,1008.3,1004.8,8,8,22.5,25.5,Yes,Yes


In [245]:
# this is where we will do hot coding to covert categorical variables to binary ones

df_sydney_processed = pd.get_dummies(data=df, columns=['RainToday', 'WindGustDir', 'WindDir9am', 'WindDir3pm'])

In [246]:
# now we change the columns to a categorical cloumn from a binary one

df_sydney_processed.replace(['No', 'Yes'], [0,1], inplace=True)

In [247]:
# train and test the data set
df_sydney_processed.drop('Date', axis=1, inplace=True)
df_sydney_processed = df_sydney_processed.astype(float)

In [248]:
features = df_sydney_processed.drop(columns='RainTomorrow', axis=1)
Y = df_sydney_processed['RainTomorrow']

In [249]:
x = np.array(df_sydney_processed)


#### Linear Regression

In [250]:
from sklearn import preprocessing
x= preprocessing.StandardScaler().fit(x).transform(x)


In [251]:
# Q1
# train test split the features with test size .2 and random state 10

x_train, x_test, Y_train, Y_test = train_test_split(x, Y, test_size=.2, random_state=10)
print(x_train.shape, Y_train.shape)
print(x_test.shape, Y_test.shape)

(2616, 67) (2616,)
(655, 67) (655,)


In [252]:
df_sydney_processed.head()

,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustSpeed,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,...,WindDir3pm_NNW,WindDir3pm_NW,WindDir3pm_S,WindDir3pm_SE,WindDir3pm_SSE,WindDir3pm_SSW,WindDir3pm_SW,WindDir3pm_W,WindDir3pm_WNW,WindDir3pm_WSW
0,19.5,22.4,15.6,6.2,0.0,41.0,17.0,20.0,92.0,84.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,19.5,25.6,6.0,3.4,2.7,41.0,9.0,13.0,83.0,73.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,21.6,24.5,6.6,2.4,0.1,41.0,17.0,2.0,88.0,86.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,20.2,22.8,18.8,2.2,0.0,41.0,22.0,20.0,83.0,90.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,19.7,25.7,77.4,4.8,0.0,41.0,11.0,6.0,88.0,74.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [253]:
Y[0:5]

0    1.0
1    1.0
2    1.0
3    1.0
4    1.0
Name: RainTomorrow, dtype: float64

In [254]:
# Q2

LinearReg = LinearRegression()
LinearReg.fit(x_train, Y_train)

LinearRegression()

In [255]:
# Q3

predictions = LinearReg.predict(x_test)

In [256]:
# Q4
from sklearn.metrics import r2_score
x_test = np.asanyarray(predictions)
Y_test = predictions
Y_test = np.asanyarray(predictions)

print('LinearRegression_MAE = %.2f' % np.mean(np.absolute(Y_test - Y_test)))
print('LinearRegression_R2 = %2f' % r2_score(Y_test, Y_test))
print('LinearRegression_MSE = %.2f' % np.mean((Y_test - Y_test)))


LinearRegression_MAE = 0.00
LinearRegression_R2 = 1.000000
LinearRegression_MSE = 0.00


In [257]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
LinearRegression_MAE =  mean_squared_error(predictions, Y_test)
LinearRegression_MSE = mean_absolute_error(predictions, Y_test)
LinearRegression_R2 = r2_score(predictions, Y_test)

In [258]:
# tubular format
# Q5

Report = {"Metrics":["MAE","MSE","R2"],"Result": 
    [LinearRegression_MAE,LinearRegression_MSE,LinearRegression_R2]}
pd.DataFrame(Report)

,Metrics,Result
0,MAE,0.0
1,MSE,0.0
2,R2,1.0


In [259]:
x_train, x_test, Y_train, Y_test = train_test_split(x, Y, test_size=.2, random_state=10)
print('Train set:', x_train.shape, Y_train.shape)
print('Test set:', x_test.shape, Y_test.shape)

Train set: (2616, 67) (2616,)
Test set: (655, 67) (655,)


In [260]:
scaler = preprocessing.StandardScaler().fit(x_train)

In [261]:
# transforming the train model and changing it to a float

x_train = scaler.transform(x_train.astype(float))
x_train[0:2]

array([[-0.01652033, -0.22351516,  3.05624855, -0.3528601 , -1.49917863,
         0.81280282,  0.54686322,  0.63906113,  1.45139438, -0.34685075,
         1.42955371,  1.46816932,  1.45395941,  1.16411721, -0.42868452,
         0.06119933,  1.71284401, -1.73381821,  1.73381821, -0.19412228,
        -0.26363085, -0.20951051, -0.09209294, -0.23052136, -0.22306777,
        -0.12773808, -0.151901  , -0.2296003 , -0.14380157,  3.64624217,
        -0.23417666, -0.12773808, -0.88431688, -0.1596252 , -0.20347303,
        -0.21540659, -0.15708867, -0.21050268, -0.18101787, -0.15836152,
        -0.16702646, -0.18325819, -0.1694292 , -0.21540659, -0.17646286,
        -0.19412228, -0.2413554 , -0.13816088,  1.26506823, -0.41667788,
        -0.17061927, -0.48850421, -0.35035237,  2.94769213, -0.13525922,
        -0.33219954, -0.14654687, -0.1077076 , -0.13671719, -0.28615705,
        -0.26196842, -0.28927244, -0.21830526, -0.11475228, -0.25777751,
        -0.24312407, -0.18436945],
       [-1.48410

In [262]:
# Creating a KNN model called KNN using the perameter 4 as k
#Q6

from sklearn.neighbors import KNeighborsClassifier 

k=4
KNN = KNeighborsClassifier(n_neighbors=k).fit(x_train, Y_train)
KNN

KNeighborsClassifier(n_neighbors=4)

In [263]:
x_test = scaler.transform(x_test.astype(float))
x_test[0:3]


array([[ 0.83774748,  2.84462403, -0.30146942,  0.30126676,  0.68061979,
         1.47343511, -0.30569589, -0.29818758, -1.59551595, -1.90097783,
        -1.41405301, -1.18749715, -0.51659173,  0.34194372,  1.58120779,
         2.19333264, -0.58382433,  0.57676174, -0.57676174, -0.19412228,
        -0.26363085, -0.20951051, -0.09209294, -0.23052136, -0.22306777,
        -0.12773808, -0.151901  , -0.2296003 , -0.14380157, -0.27425496,
         4.27028033, -0.12773808, -0.88431688, -0.1596252 , -0.20347303,
        -0.21540659, -0.15708867, -0.21050268, -0.18101787, -0.15836152,
        -0.16702646, -0.18325819, -0.1694292 , -0.21540659, -0.17646286,
        -0.19412228, -0.2413554 , -0.13816088, -0.7904712 ,  2.3999354 ,
        -0.17061927, -0.48850421, -0.35035237,  2.94769213, -0.13525922,
        -0.33219954, -0.14654687, -0.1077076 , -0.13671719, -0.28615705,
        -0.26196842, -0.28927244, -0.21830526, -0.11475228, -0.25777751,
        -0.24312407, -0.18436945],
       [ 0.02728

In [264]:
# Use the predict method on the testing data x_test,
# save it to predictions

#Q7

predictions = KNN.predict(x_test)

In [265]:
# Use the predictions and the Y_test to calculate the accuracy_score,
# the jaccard_score, and the f1_score.

#Q8

from sklearn.metrics import jaccard_score, f1_score, accuracy_score

KNN_Accuracy_Score = accuracy_score(predictions, Y_test)
KNN_JaccardIndex = jaccard_score(predictions, Y_test)
KNN_F1_Score = f1_score(predictions, Y_test)

In [266]:
Report = {"Metrics":["Accuracy_Score","Jaccard_Score","F1_Score"],"Result": 
    [KNN_Accuracy_Score,KNN_JaccardIndex,KNN_F1_Score]}
pd.DataFrame(Report)

,Metrics,Result
0,Accuracy_Score,0.816794
1,Jaccard_Score,0.378238
2,F1_Score,0.548872


In [267]:
# create a decision tree model called tree, 
# use data x_test and Y_test

#Q9
from sklearn.tree import DecisionTreeClassifier
Tree = DecisionTreeClassifier(criterion = 'entropy', max_depth=4)
Tree.fit(x_train, Y_train)

DecisionTreeClassifier(criterion='entropy', max_depth=4)

In [268]:
# use the prdict method testing dat x_test save it to predictions

#Q10

predictions = Tree.predict(x_test)

In [269]:
# use predictions and the y_test dataframe calculate the value for each metric

#Q11

Tree_Accuracy_Score = accuracy_score(predictions,Y_test)
Tree_JaccardIndex = jaccard_score(predictions, Y_test)
Tree_F1_Score = f1_score(predictions, Y_test)

In [270]:
# Making it tubular

Report = {'Mertics': ['Accuarcy_Score', 'JaccardIndex', 'F1_Score'], 'Result':
    [Tree_Accuracy_Score, Tree_JaccardIndex, Tree_F1_Score]}
pd.DataFrame(Report)
          

,Mertics,Result
0,Accuarcy_Score,1.0
1,JaccardIndex,1.0
2,F1_Score,1.0


In [271]:
# Use train_test_slpit to split the features and Y dataframes,
# with a test_size of .2 and random state 1

#Q12

from sklearn.model_selection import train_test_split
x_train, x_test, Y_train, Y_test = train_test_split(x, Y, test_size=.2, random_state=10)
print('Train set:', x_train.shape, Y_train.shape)
print('Test set:', x_test.shape, Y_test.shape)

Train set: (2616, 67) (2616,)
Test set: (655, 67) (655,)


In [272]:
# train logistic regression using the train data x_train, y_train,
# solver = liblinear. 

#Q13

LR = LogisticRegression(C=0.01, solver='liblinear').fit(x_train, Y_train)
LR

LogisticRegression(C=0.01, solver='liblinear')

In [273]:
# use the predictions and predict_proba using x_test,
# save 2 arrays as predictions and predict_proba

#Q14

predictions = LR.predict(x_test)
predict_proba = LR.predict_proba(x_test)

In [280]:
#use predictions adn predict_proba with the Y_test DFrame, 
# to calculate the value for each metric using the righ function

#Q15

LR_Accuracy_Score = accuracy_score(predictions, Y_test)
LR_JaccardIndex = jaccard_score( predictions, Y_test)
LR_F1_Score = f1_score(predictions, Y_test)
LR_Log_Loss = log_loss(predictions, Y_test)


In [281]:
# making it tubular

Report = {'Metrics': ['Accuracy_score', 'JaccardIndex', 'F1_score', 'Log_loss'], 'Results':
          [LR_Accuracy_Score, LR_JaccardIndex, LR_F1_Score, LR_Log_Loss]}
pd.DataFrame(Report)

,Metrics,Results
0,Accuracy_score,0.993893
1,JaccardIndex,0.978261
2,F1_score,0.989011
3,Log_loss,0.210929


In [276]:
# create and train a svm model using x_train and Y_train

#Q16

SVM = svm.SVC(kernel='rbf')
SVM.fit(x_train, Y_train)

SVC()

In [277]:
# Use theprdict method on the testing data x_test and save it as predictions

#Q17

predictions = SVM.predict(x_test)

In [278]:
# using predictions and the y_test calculate the value for each metric 

#Q18

SVM_Accuracy_Score = accuracy_score(predictions, Y_test)
SVM_JaccardIndex = jaccard_score(predictions, Y_test)
SVM_F1_Score = f1_score(predictions, Y_test)

In [279]:
Reports = {'Metrics': ['Accuracy_Score', 'JaccardIndex', 'F1_Score'], 'Results':
          [SVM_Accuracy_Score, SVM_JaccardIndex, SVM_F1_Score]}
pd.DataFrame(Reports)
          

,Metrics,Results
0,Accuracy_Score,0.993893
1,JaccardIndex,0.978261
2,F1_Score,0.989011
